In [2]:
import json
import logging
import sys

import requests

sys.path.append("/path/to/the/project")  # TODO: set correct path to bte-py project, so we can import its modules.

from biothings_explorer.call_apis_v2.query import SmartAPI
from biothings_explorer.call_apis_v2.query_validator import InvalidQuery

logging.getLogger('urllib3').setLevel(logging.ERROR)
logging.getLogger('httpx').setLevel(logging.ERROR)
logging.basicConfig(filename='myapp.log', level=logging.INFO)
logger = logging.getLogger(__name__)


smart_api = SmartAPI("https://raw.githubusercontent.com/NCATS-Tangerine/translator-api-registry/master/mygene.info/openapi_full.yml")

# smart_api = SmartAPI("https://raw.githubusercontent.com/NCATS-Tangerine/translator-api-registry/master/semmeddb/smartapi.yaml")
# smart_api = SmartAPI("https://raw.githubusercontent.com/NCATS-Tangerine/translator-api-registry/master/dgidb/openapi.yml")
# smart_api = SmartAPI("https://raw.githubusercontent.com/NCATS-Tangerine/translator-api-registry/master/biothings_foodb/smartapi.yaml")
# smart_api = SmartAPI("https://raw.githubusercontent.com/NCATS-Tangerine/translator-api-registry/master/ddinter/ddinter.yaml")
# smart_api = SmartAPI("https://raw.githubusercontent.com/NCATS-Tangerine/translator-api-registry/master/pfocr/smartapi.yaml")

# smart_api = SmartAPI("http://www.smart-api.info/api/metadata/e8818d98f59fb98ad5ba6e4637b5dec4")

all_edges = smart_api.list_metakg()

def get_ids(edge, limit=3):
    query_operation = edge["bte"]["query_operation"]
    url = query_operation["server"] + query_operation["path"]
    params = query_operation["params"]
    resp = requests.get(url, params)
    data = resp.json()
    ids = [record["_id"] for record in data["hits"]]
    return ids[:limit]



In [7]:
test_edges = all_edges

# test using get_edge
for edge in test_edges[-1:]:
    logger.info(f"Process edge: {json.dumps(edge)}")
    count_not_found = 0
    ids = get_ids(edge)
    for _id in ids:
        try:
            resp_data = smart_api.get_edge(edge, _id)
            if resp_data:
                for record in resp_data:
                    print(record)
            else:
                count_not_found += 1
                if count_not_found > 10:
                    break
        except InvalidQuery:
            continue
        except Exception as ex:
            logger.error(f"Id: {_id}, ex: {ex}", exc_info=True)

# Test using get_edges
# for edge in test_edges[5:]:
#     logger.info(f"Process edge: {json.dumps(edge)}")
#     count_not_found = 0
#     ids = get_ids(edge, limit=100)
#     try:
#         resp_data = smart_api.get_edges(edge, ids, batch_size=55)
#         for sub in resp_data:
#             print(sub)
#     except InvalidQuery:
#         continue
#     except Exception as ex:
#         logger.error(f"Id: {ids}, ex: {ex}", exc_info=True)